In [2]:
pip install -r requirements.txt

  Using cached ultralytics-8.3.160-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached pillow-11.2.1-cp311-cp311-win_amd64.whl.metadata (9.1 kB)
  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached scipy-1.16.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.7.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cache


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# from dotenv import load_dotenv
# import os
# import requests

# # Load .env values
# load_dotenv()
# BOT_TOKEN = os.getenv("BOT_TOKEN")
# CHAT_ID = os.getenv("CHAT_ID")

# # Send test message
# url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
# data = {
#     "chat_id": CHAT_ID,
#     "text": "✅ Bot setup successful! Helmet violation system is ready."
# }
# response = requests.post(url, data=data)

# # Print API response
# print(response.json())


{'ok': True, 'result': {'message_id': 5, 'from': {'id': 7850495636, 'is_bot': True, 'first_name': 'Traffic Violation Ahmedabad 🚦', 'username': 'Traffic_Violation_Ahmedabad_Bot'}, 'chat': {'id': 808101216, 'first_name': 'Shorya Sharma', 'username': 'Shorya22', 'type': 'private'}, 'date': 1751185199, 'text': '✅ Bot setup successful! Helmet violation system is ready.'}}


In [43]:
import cv2
import time
import os
import requests
from dotenv import load_dotenv
from ultralytics import YOLO

# Load environment variables from .env file
load_dotenv()
BOT_TOKEN = os.getenv("BOT_TOKEN")
CHAT_ID = os.getenv("CHAT_ID")

# Create directory for saved violations
SAVE_DIR = "violations"
os.makedirs(SAVE_DIR, exist_ok=True)

# Load YOLOv8 model
model = YOLO("Helmet_Violation_Detection.pt")

# Start webcam
cap = cv2.VideoCapture("samplev1.mp4")

# Function to send Telegram photo alert
def send_telegram_alert(image_path, caption="🚨 Helmet Violation Detected!"):
    try:
        url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendPhoto"
        if not os.path.exists(image_path):
            print(f"❌ Image not found: {image_path}")
            return

        with open(image_path, "rb") as photo:
            files = {"photo": photo}
            data = {"chat_id": CHAT_ID, "caption": caption}
            response = requests.post(url, files=files, data=data)
            print("📤 Telegram Response:", response.status_code, response.json())

    except Exception as e:
        print("❗ Error sending Telegram alert:", str(e))


print("🚦 Helmet Violation Detection is Running...")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame,(640,380))
    if not ret:
        break

    # Run YOLO inference
    results = model(frame)[0]
    names = model.names

    found_rider = False
    found_helmet = False
    found_violation = False

    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = names[cls_id]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        print(f"🧠 Detected: {label} | Confidence: {conf:.2f}")

        if conf < 0.70:
            continue

        color = (0, 255, 0)

        if label == "rider":
            found_rider = True
        elif label == "with helmet":
            found_helmet = True
        elif label == "without helmet":
            found_violation = True
            color = (0, 0, 255)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


    # If violation detected (rider without helmet)
    if found_rider and found_violation and not found_helmet:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(SAVE_DIR, f"violation_{timestamp}.jpg")
        cv2.imwrite(img_path, frame)
        time.sleep(0.5)  # Ensure file is saved before sending
        send_telegram_alert(img_path)
        print(f"🚨 Helmet Violation Sent at {timestamp}")

    # Show webcam preview
    cv2.imshow("Helmet Violation Detection", frame)
    if cv2.waitKey(33) & 0xFF == ord("q"):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


🚦 Helmet Violation Detection is Running...

0: 384x640 2 without helmets, 2 riders, 436.4ms
Speed: 3.9ms preprocess, 436.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
🧠 Detected: rider | Confidence: 0.87
🧠 Detected: without helmet | Confidence: 0.85
🧠 Detected: without helmet | Confidence: 0.75
🧠 Detected: rider | Confidence: 0.62
📤 Telegram Response: 200 {'ok': True, 'result': {'message_id': 90, 'from': {'id': 7850495636, 'is_bot': True, 'first_name': 'Traffic Violation Ahmedabad 🚦', 'username': 'Traffic_Violation_Ahmedabad_Bot'}, 'chat': {'id': 808101216, 'first_name': 'Shorya Sharma', 'username': 'Shorya22', 'type': 'private'}, 'date': 1751188901, 'photo': [{'file_id': 'AgACAgUAAxkDAANaaGEFpWMe4MZxO9_zSJ9gomS4hHsAAqDDMRuDJglX4axGo4MROEcBAAMCAANzAAM2BA', 'file_unique_id': 'AQADoMMxG4MmCVd4', 'file_size': 1340, 'width': 90, 'height': 53}, {'file_id': 'AgACAgUAAxkDAANaaGEFpWMe4MZxO9_zSJ9gomS4hHsAAqDDMRuDJglX4axGo4MROEcBAAMCAANtAAM2BA', 'file_unique_id': 'AQADoMMx